In [1]:
from pyspark import SparkConf , SparkContext
import binascii
import time

第一個mapper，將word從文章裡分開，且只要開頭為英文或是數字的word，key為1，value為word

In [2]:
def mapper1(line):
    wordlist = line.split(" ")
    maplist = []
    for word in wordlist:
        if word != "":
            if( (ord(word[0]) >= 65 and ord(word[0]) <= 90) or (ord(word[0]) >= 97 and ord(word[0]) <= 122) or  (ord(word[0]) >= 48 and ord(word[0]) <= 57)):
                maplist.append((1,[word]))
    return maplist

第一個reducer，直接return x+y

In [3]:
def reducer1(x,y):
    return x+y

第二個mapper，將三個三個單字利用空白格串接在一起，以符合題目要求。三個單字串在一起之後，為了方便計算，先將其encode()，再轉成binascii.crc32，如此一來，可以將字串轉成數字。

In [4]:
def mapper2(line):
    maplist = []
    i = 0
    for word in line[1]:
        # may not use " "
        stringshingle = line[1][i]+" "+line[1][i+1]+" "+line[1][i+2]
        stringshingle = stringshingle.encode()
        maplist.append(binascii.crc32(stringshingle))
        i = i + 1
        if(i == (len(line[1]) - 2)):
            break
    return maplist

利用for迴圈來map題目的101個文章，shinglesInDocInts記錄的是文章編號以及所有剛剛轉成數字的字串，並且將其sort好，且每個字串只會記錄一次，即不會重複計算，在每圈for迴圈的最後將其記錄在docsAsShingleRDD裡，也就是說當for迴圈結束時，docsAsShingleRDD會是一個二維陣列，index i 存的會是文章編號，index j 存的會是shinglesInDocInts。而docsAsShingleList記錄的是101篇文章，所有的字串組合，並且將其sort好，類似索引的概念，這樣做是為了計算總共有幾個字串，然後該字串應該在List中的哪個位置，且每個值只會記錄一次，即不會重複計算。

In [5]:
tStart = time.time()
conf=SparkConf().set("spark.default.parallelism", 4)
sc = SparkContext(conf=conf)
docsAsShingleRDD = [0] * 101
docsAsShingleList = []
for i in range(0,101):
    shinglesInDocInts = []
    if (i >= 0 and i <= 8):
        stringtxt = "00" + str(i+1) + ".txt"
    elif(i >= 9 and i <= 98):
        stringtxt = "0" + str(i+1) + ".txt"
    else:
        stringtxt = str(i+1) + ".txt"
    lines1 = sc.textFile(stringtxt).flatMap(mapper1).reduceByKey(reducer1).flatMap(mapper2)
    lenlines = lines1.count()
    hashvalue = lines1.take(lenlines)
    shinglesInDocInts.append(i+1)
    for j in range(lenlines):
        if hashvalue[j] not in shinglesInDocInts:
            shinglesInDocInts.append(hashvalue[j])
            if hashvalue[j] not in docsAsShingleList:
                docsAsShingleList.append(hashvalue[j])
    shinglesInDocInts.sort()
    docsAsShingleRDD[i] = shinglesInDocInts
    #print(i,docsAsShingleList[i])
#print(docsAsShingleList)
docsAsShingleList.sort()
#print(docsAsShingleRDD)
tEnd2 = time.time()
print(tEnd2 - tStart)
            

31.47405171394348


第三個mapper，key為文章編號，也就是剛剛存在line[0]的值，value為該字串在docsAsShingleList中的index。

In [6]:
def mapper3(line):
    maplist = []
    for i in range(1,len(line)):
        maplist.append((line[0],[docsAsShingleList.index(line[i])]))
    return maplist

將剛剛做好的docsAsShingleRDD轉成RDD，也就是lines_mh，然後使用mapper3來map，再使用reducer1來reduce，如此一來，就會知道每個字串組合在docsAsShingleList中的index。

In [7]:
lines_mh = sc.parallelize(docsAsShingleRDD)
lines_mh = lines_mh.flatMap(mapper3)
lines_mh = lines_mh.reduceByKey(reducer1)


接著計算docsAsShingleList有多長，用於等等的hash。

In [8]:
len_docsAsShingleList = len(docsAsShingleList)

第四個mapper，用來hash，100個hash function的通式為((j + 1) * line[1][i] + j )% len_docsAsShingleList，j為0~99，代表100個hash function，hold與hnew陣列用來計算最後的hash值，最後key為文章編號，也就是line[0]，value為最終hash值，也就是hold。

In [9]:
def mapper4(line):
    maplist = []
    hold = [len_docsAsShingleList+2] * 100
    hnew = [len_docsAsShingleList+2] * 100
    for i in range(len(line[1])):
        for j in range(100):
            hnew[j] = ( ((j + 1) * line[1][i] + j )% len_docsAsShingleList)  
            if(hnew[j] < hold[j]):
                hold[j] = hnew[j]
    maplist.append((line[0],hold))
    return maplist

lines_mh使用mapper4。

In [10]:
lines_mh = lines_mh.flatMap(mapper4)


將所有字串的hash值存成陣列sig_matrix，方便之後計算similarity。

In [11]:
shingle_matrix = lines_mh.take(101)
sig_matrix = [0] * 101
for i in range(101):
    sig_matrix[shingle_matrix[i][0]-1] = shingle_matrix[i][1]
#print(sig_matrix)

第五個mapper用來做LSH，將每篇文章的100個hash值，分成50個band且r = 2，bucket則是利用2 * 第一個r + 5 * 第二個r，最後key為(band的number,bucket的number)，value則為文章編號。

In [12]:
def mapper5(line):
    maplist = []
    for i in range(50):
        band_num = i + 1
        bucket_num = 2*line[1][i*2] + 5*line[1][i*2+1] 
        maplist.append(((band_num,bucket_num),[line[0]]))
    return maplist

lines_mh先後使用mapper5與reducer1。

In [13]:
lines_mh = lines_mh.flatMap(mapper5)
lines_mh = lines_mh.reduceByKey(reducer1)


第六個mapper，將hash到相同bucket完，只有一個文章的丟掉。

In [14]:
def mapper6(line):
    maplist = []
    if (len(line[1]) != 1):
        maplist.append((line[0],line[1]))
    return maplist

lines_mh使用mapper6。

In [15]:
lines_mh = lines_mh.flatMap(mapper6)

第七個mapper，將所有的candidate pair，利用剛剛存起來的sig_matrix，計算similarity。key為candidate pair，value為similarity。

In [16]:
def mapper7(line):
    maplist = []
    for i in range(len(line[1])):
        for j in range(len(line[1])):
            matchcount = 0
            if(j > i):
                for k in range(100):
                    if(sig_matrix[line[1][i]-1][k] == sig_matrix[line[1][j]-1][k]):
                        matchcount = matchcount + 1
                maplist.append(((line[1][i],line[1][j]),matchcount/100))
    return maplist

lines_mh使用mapper7

In [17]:
lines_mh = lines_mh.flatMap(mapper7)
#print(lines_mh.collect())

第二個reducer，因為同一組candidate pair可能出現多次，因此只要保留一次，利用直接return x來完成。

In [18]:
def reducer2(x,y):
    return x

lines_mh使用reducer2

In [19]:
lines_mh = lines_mh.reduceByKey(reducer2)
#print(lines_mh.collect())

第八個mapper，將key設為similarity，value設為candidate pair，以便待會利用sortByKey來排名。

In [20]:
def mapper8(line):
    maplist = []
    maplist.append((line[1],[line[0][0],line[0][1]]))
    return maplist

lines_mh使用mapper8，並且依照similarity，利用sortByKey由大到小排名。

In [21]:
lines_mh = lines_mh.flatMap(mapper8)
lines_mh = lines_mh.sortByKey(ascending= False)

將結果印出，資料夾裡也有一份outputfile，麻煩助教了。

In [22]:
answer = lines_mh.take(10)
for i in range(10):
    print("Top",(i+1),answer[i][1][0],"and",answer[i][1][1],":",answer[i][0])
tEnd = time.time()
print(tEnd - tStart)

Top 1 12 and 20 : 1.0
Top 2 52 and 84 : 1.0
Top 3 30 and 35 : 0.82
Top 4 49 and 47 : 0.7
Top 5 40 and 14 : 0.56
Top 6 48 and 49 : 0.56
Top 7 38 and 23 : 0.51
Top 8 88 and 49 : 0.45
Top 9 48 and 47 : 0.41
Top 10 88 and 47 : 0.29
35.77479076385498


In [138]:
sc.stop()